In [1]:
from htm.bindings.sdr import SDR, Metrics
from htm.encoders.date import DateEncoder
from htm.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood
import numpy as np
import pandas as pd
import pathlib
import datetime
import csv
from datetime import datetime
import os
from htm.encoders.rdse import RDSE, RDSE_Parameters
import time
import traceback
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn_extra.cluster import KMedoids
from tqdm import tqdm

In [2]:
class ReflexiveMemory:
  def __init__(self, reflexSize):
    self.acKey0 = None
    self.pairs = {}
    self.anomalyRM = []
    self.anomalyTM = []
    self.enableLearn = False
    self.tableSize  = reflexSize
    self.historyRM = []
    self.historyTM = []
    self.historyGT = []

  def add(self, denseColumns):
    acKey1 = '-'.join(map(str, denseColumns.sparse))
    if(self.acKey0 != None):

      sequence = self.pairs.get(self.acKey0, {})
      sequence_data = sequence.get(acKey1, {
         "count": 0,
         "time": datetime.now()
      })
      sequence_data["count"] = sequence_data["count"] + 1
      sequence_data["time"] = datetime.now()

      if self.pairs.get(self.acKey0, None) is None:
        self.pairs[self.acKey0] = { acKey1: sequence_data }
      else:
        self.pairs[self.acKey0][acKey1] = sequence_data

      table_entries = 0
      oldKey1 = None
      oldKey2 = None
      oldTime = datetime.now()
      for key1, value1 in self.pairs.items():
        table_entries = table_entries + len(value1.items())
        for key2, value2 in value1.items():
          if value2['time'] < oldTime:
            oldKey1 = key1
            oldKey2 = key2
            oldTime = datetime.now()
      if table_entries > self.tableSize:
        self.enableLearn = True
        del self.pairs[oldKey1][oldKey2]
        if len(self.pairs[oldKey1].items()) == 0:
          del self.pairs[oldKey1]

    self.acKey0 = acKey1

  def predict(self, denseColumns):
    return_count = 0
    return_sdr = None

    acKey = '-'.join(map(str, denseColumns.sparse))
    sequences = self.pairs.get(acKey, {})
    for sequence_key, sequence_data in sequences.items():
      if sequence_data["count"] > return_count:
        return_count = sequence_data["count"]
        return_sdr = sequence_key

    if return_sdr is not None:
      tmp_sdr = SDR( denseColumns.dimensions )
      tmp_sdr.sparse = list(map(int, return_sdr.split('-')))
      return_sdr = tmp_sdr
    else:
      return_count = None

    return return_count, return_sdr

  def learn(self, denseColumns1, sp, tm):

    tm.activateDendrites(True)
    predictiveColumns = SDR( tm.getPredictiveCells().dimensions[0] )
    predictiveColumns.sparse = list(set(sorted(list(np.where(tm.getPredictiveCells().dense == 1)[0]))))

    activeColumns1 = SDR( predictiveColumns.dimensions )
    sp.compute(denseColumns1, False, activeColumns1)

    denseColumns0 = SDR( denseColumns1.dimensions )
    denseColumns0.sparse = list(map(int, self.acKey0.split('-')))
    activeColumns0 = SDR( predictiveColumns.dimensions )
    sp.compute(denseColumns0, False, activeColumns0)
 
    reflexiveCount, denseReflexiveColumns = self.predict(denseColumns0)
    reflexiveColumns = SDR( predictiveColumns.dimensions )
    if denseReflexiveColumns is not None:
      sp.compute(denseReflexiveColumns, False, reflexiveColumns)    

    overlapRM = 1 - np.count_nonzero((reflexiveColumns.dense & activeColumns1.dense)) / np.count_nonzero(activeColumns1.dense)
    self.anomalyRM.append( overlapRM )

    overlapTM = 1 - np.count_nonzero((predictiveColumns.dense & activeColumns1.dense)) / np.count_nonzero(activeColumns1.dense)
    self.anomalyTM.append( overlapTM )

    self.historyRM.append( reflexiveColumns.dense )
    self.historyTM.append( predictiveColumns.dense )
    self.historyGT.append( activeColumns1.dense )

  def compute(self, denseColumns, sp, tm):
    if self.acKey0 is not None:
      self.learn(denseColumns, sp, tm)  
    self.add(denseColumns)
  

In [3]:
inputSources = [
   "monthly_sp500.csv",
   "weekly_dow_jones.csv",
   "weekly_nasdaq.csv",
   "weekly_sp500.csv",
   "monthly_vix_close.csv",
   "monthly_vix_high.csv",
   "monthly_vix_low.csv",
   "monthly_vix_open.csv",
   "daily_natural_gas.csv",
   "daily_oil_prices.csv",
   "value1_vix_close.csv",
   "value1_vix_high.csv",
   "value1_vix_low.csv",
   "value1_vix_open.csv"
]

In [4]:

config = {
    'enc': {
        "value" :
            {'resolution': 0.88, 'size': 700, 'sparsity': 0.02},
        "time": 
            {'timeOfDay': (30, 1), 'weekend': 21}
    },
    'sp': {
        'inputDimensions': None,
        'columnDimensions': (1638,),
        'potentialPct': 0.85,
        'potentialRadius': None,
        'globalInhibition': True,
        'localAreaDensity': 0.04395604395604396,
        'synPermInactiveDec': 0.006,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'boostStrength': 3.0,
        'wrapAround': True,
        'seed': 1,
        'learn': False,
    },
    'tm': {
        'cellsPerColumn': 13,
        'activationThreshold': 17,
        'initialPermanence': 0.21,
        'minThreshold': 10,
        'maxNewSynapseCount': 32,
        'permanenceIncrement': 0.1,
        'permanenceDecrement': 0.1,
        'predictedSegmentDecrement': 0.0,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'learn': True
    },
    'anomaly': {'period': 1000},
    'learnRows': 100,
    'reflexSize': 128
}


In [5]:
def anomalyScore(y, x):
    if np.count_nonzero(y) != 0:
        return 1 - np.count_nonzero((x & y)) / np.count_nonzero(y)
    return 1

def match(y, x, idx1):
  overlap_min = 0.02
  n_samples = len(y)
  score1 = anomalyScore(y[idx1], x[idx1])
  if score1 > (1 - overlap_min):
     idx_closest = None
     score_closest = None
     for idx2 in range(n_samples):
        score2 = anomalyScore(y[idx2], x[idx1])
        if score_closest is None or score_closest > score2:
            score_closest = score2
            idx_closest = idx2
     return idx_closest
  return idx1

In [6]:
input_path = pathlib.Path('../datasets/numenta')
dataset_metrics = []

pbar = tqdm(total=len(inputSources))
for dataset in inputSources:

    tm_infer_tm = 0
    tm_infer_rm = 0

    records = []
    with open(input_path.joinpath(dataset), "r") as fin:
        reader = csv.reader(fin)
        headers = next(reader)
        next(reader)
        next(reader)
        for record in reader:
            records.append(record)
        
    scalarEncoderParams = RDSE_Parameters()
    scalarEncoderParams.size = config["enc"]["value"]["size"]
    scalarEncoderParams.sparsity = config["enc"]["value"]["sparsity"]
    scalarEncoderParams.resolution = config["enc"]["value"]["resolution"]
    scalarEncoder = RDSE( scalarEncoderParams )
    encodingWidth = (scalarEncoder.size)

    config['sp']['inputDimensions'] = (encodingWidth,)
    config['sp']['potentialRadius'] = encodingWidth

    sp = SpatialPooler(
        inputDimensions = config['sp']['inputDimensions'],
        columnDimensions = config['sp']['columnDimensions'],
        potentialPct = config['sp']['potentialPct'],
        potentialRadius = config['sp']['potentialRadius'],
        globalInhibition = config['sp']['globalInhibition'],
        localAreaDensity = config['sp']['localAreaDensity'],
        synPermInactiveDec = config['sp']['synPermInactiveDec'],
        synPermActiveInc = config['sp']['synPermActiveInc'],
        synPermConnected = config['sp']['synPermConnected'],
        boostStrength = config['sp']['boostStrength'],
        wrapAround = config['sp']['wrapAround'],
        seed = config['sp']['seed']
    )

    tm = TemporalMemory(
        columnDimensions = config['sp']['columnDimensions'],
        cellsPerColumn = config['tm']['cellsPerColumn'],
        activationThreshold = config['tm']['activationThreshold'],
        initialPermanence = config['tm']['initialPermanence'],
        connectedPermanence = config['sp']['synPermConnected'],
        minThreshold = config['tm']['minThreshold'],
        maxNewSynapseCount = config['tm']['maxNewSynapseCount'],
        permanenceIncrement = config['tm']['permanenceIncrement'],
        permanenceDecrement = config['tm']['permanenceDecrement'],
        predictedSegmentDecrement = config['tm']['predictedSegmentDecrement'],
        maxSegmentsPerCell = config['tm']['maxSegmentsPerCell'],
        maxSynapsesPerSegment = config['tm']['maxSynapsesPerSegment']
    )

    rm = ReflexiveMemory( config['reflexSize'] )

    enc_info = Metrics( [encodingWidth], 999999999)
    sp_info = Metrics( sp.getColumnDimensions(), 999999999 )
    tm_info = Metrics( [tm.numberOfCells()], 999999999 )
    anomaly_history = AnomalyLikelihood(config["anomaly"]["period"])

    inputs = []
    anomaly = []
    anomalyProb = []
   
    try:
        
        for count, record in enumerate(records):

            consumption = float(record[1])

            inputs.append( consumption )
            consumptionBits = scalarEncoder.encode(consumption)

            encoding = SDR( consumptionBits )
            enc_info.addData( encoding )
            
            activeColumns = SDR( sp.getColumnDimensions() )

            learn_sp = config['sp']['learn']
            learn_tm = config['tm']['learn']
            if count < config['learnRows']:
                learn_sp = True
                learn_tm = True

            rm.compute(encoding, sp, tm)

            sp.compute(encoding, learn_sp, activeColumns)
            sp_info.addData( activeColumns )

            tmp_tm = time.time()
            tm.compute(activeColumns, learn=learn_tm)
            tm_info.addData( tm.getActiveCells().flatten() )
            tm_infer_tm = tm_infer_tm + (time.time() - tmp_tm)

            tmp_tm = time.time()
            rm.predict(encoding)
            tm_infer_rm = tm_infer_rm + (time.time() - tmp_tm)

            anomaly.append( tm.anomaly )
            anomalyProb.append( anomaly_history.compute(tm.anomaly) )

    except Exception as e:
        print(traceback.format_exc())
        print(e)

    metric = {}
    metric['dataset'] = dataset

    n_samples = len(rm.historyGT)

    Y_labels = list(range(n_samples))
    X_labels = [ match(rm.historyGT, rm.historyRM, idx) for idx in range(n_samples)]
    precision, recall, fscore, support = precision_recall_fscore_support(Y_labels, X_labels, average='macro', zero_division=0.0)
    metric['total-infe-time-rm'] = tm_infer_rm
    metric['infer-time-rm'] = tm_infer_rm / len(rm.anomalyRM)
    metric['anomaly-avg-rm'] = sum(rm.anomalyRM) / len(rm.anomalyRM)
    metric['anomaly-samples-rm'] = len(rm.anomalyRM)
    metric['accuracy0-rm'] = accuracy_score(Y_labels, X_labels)
    metric['precision0-rm'] = precision
    metric['recall0-rm'] = recall
    metric['fscore0-rm'] = fscore
    metric['support0-rm'] = support
    
    X = pairwise_distances(rm.historyGT, metric=anomalyScore)
    kmedoids = KMedoids(n_clusters=12, random_state=0).fit(X)
    X_labels = kmedoids.labels_
    Y = pairwise_distances(rm.historyRM, rm.historyGT, metric=anomalyScore)
    Y_labels = kmedoids.predict(Y)
    precision, recall, fscore, support = precision_recall_fscore_support(X_labels, Y_labels, average='macro', zero_division=0.0)
    metric['accuracy1-rm'] = accuracy_score(X_labels, Y_labels)
    metric['precision1-rm'] = precision
    metric['recall1-rm'] = recall
    metric['fscore1-rm'] = fscore
    metric['support1-rm'] = support

    Y_labels = list(range(n_samples))
    X_labels = [ match(rm.historyGT, rm.historyTM, idx) for idx in range(n_samples)]
    precision, recall, fscore, support = precision_recall_fscore_support(Y_labels, X_labels, average='macro', zero_division=0.0)
    metric['total-infer-time-tm'] = tm_infer_tm
    metric['infer-time-tm'] = tm_infer_tm / len(rm.anomalyTM)
    metric['anomaly-avg-tm'] = sum(rm.anomalyTM) / len(rm.anomalyTM)
    metric['anomaly-samples-tm'] = len(rm.anomalyTM)
    metric['accuracy0-tm'] = accuracy_score(Y_labels, X_labels)
    metric['precision0-tm'] = precision
    metric['recall0-tm'] = recall
    metric['fscore0-tm'] = fscore
    metric['support0-tm'] = support

    X = pairwise_distances(rm.historyGT, metric=anomalyScore)
    kmedoids = KMedoids(n_clusters=12, random_state=0).fit(X)
    X_labels = kmedoids.labels_
    Y = pairwise_distances(rm.historyTM, rm.historyGT, metric=anomalyScore)
    Y_labels = kmedoids.predict(Y)
    precision, recall, fscore, support = precision_recall_fscore_support(X_labels, Y_labels, average='macro', zero_division=0.0)
    metric['accuracy1-tm'] = accuracy_score(X_labels, Y_labels)
    metric['precision1-tm'] = precision
    metric['recall1-tm'] = recall
    metric['fscore1-tm'] = fscore
    metric['support1-tm'] = support

    metric['nupic-anomaly-avg'] = sum(anomaly) / len(anomaly)
    metric['nupic-anomaly-prob-avg'] = np.count_nonzero(anomalyProb) / len(anomalyProb)

    dataset_metrics.append(metric)
    pbar.update(1)

pbar.close()

100%|██████████| 14/14 [51:20<00:00, 220.03s/it]


In [7]:
df = pd.DataFrame(dataset_metrics)
df.to_csv('metrics.csv', index=False)

| Variable | Source | Description | 
| :---------------- | :------ | :------ | 
| dataset | | Dataset Name | 
| precision0-rm | RM | Precision (1-to-1 Matching) | 
| recall0-rm | RM | Recall (1-to-1 Matching) | 
| fscore0-rm | RM | F-Score (1-to-1 Matching) | 
| support0-rm | RM | Support (1-to-1 Matching) | 
| accuracy0-rm | RM | Accuracy (1-to-1 Matching) | 
| precision1-rm | RM | Precision (Clustering) | 
| recall1-rm | RM | Recall (Clustering) | 
| fscore1-rm | RM | F-Score (Clustering) | 
| support1-rm | RM | Support (Clustering) | 
| accuracy1-rm | RM | Accuracy (Clustering) | 
| anomaly-avg-rm | RM | Average Anomaly (Ilia Method) | 
| anomaly-samples-rm | RM | Samples | 
| total-infe-time-rm | RM | Inference Time per input | 
| infer-time-rm | RM | Total Inference Time (all samples) | 
| precision0-tm | TM | Precision (1-to-1 Matching) |
| recall0-tm | TM | Recall (1-to-1 Matching) |
| fscore0-tm | TM | F-Score (1-to-1 Matching) |
| support0-tm | TM | Support (1-to-1 Matching) |
| accuracy0-tm | TM | Accuracy (1-to-1 Matching) |
| precision1-tm | TM | Precision (Clustering) |
| recall1-tm | TM | Recall (Clustering) |
| fscore1-tm | TM | F-Score (Clustering) |
| support1-tm | TM | Support (Clustering) |
| accuracy1-tm | TM | Accuracy (Clustering) |
| anomaly-avg-tm | TM | Average Anomaly (Ilia Method) | 
| anomaly-samples-tm | TM | Samples | 
| total-infer-time-tm | TM | Inference Time per input | 
| infer-time-tm | TM | Total Inference Time (all samples) | 
| nupic-anomaly-avg | TM | Average Anomaly (Numenta Method) |
| nupic-anomaly-prob-avg | TM | Number of Anomalous Instances (Percentage) |


In [8]:
features = [
    'dataset',
    'total-infe-time-rm',
    'infer-time-rm',
    'anomaly-avg-rm',
    'anomaly-samples-rm',
    'accuracy0-rm',
    'precision0-rm',
    'recall0-rm',
    'fscore0-rm',
    'support0-rm',
    'accuracy1-rm',
    'precision1-rm',
    'recall1-rm',
    'fscore1-rm',
    'support1-rm'
]
df[features]

,dataset,total-infe-time-rm,infer-time-rm,anomaly-avg-rm,anomaly-samples-rm,accuracy0-rm,precision0-rm,recall0-rm,fscore0-rm,support0-rm,accuracy1-rm,precision1-rm,recall1-rm,fscore1-rm,support1-rm
0,monthly_sp500.csv,0.013211,0.000007,0.506075,1765,0.559207,0.558641,0.559207,0.558642,None,0.615297,0.635762,0.540735,0.505182,None
1,weekly_dow_jones.csv,0.015027,0.000007,0.993867,2079,0.018759,0.016635,0.018759,0.017108,None,0.097643,0.062958,0.087088,0.023132,None
2,weekly_nasdaq.csv,0.016087,0.000008,0.976522,2080,0.042788,0.042308,0.042788,0.042308,None,0.094231,0.161391,0.093936,0.035844,None
3,weekly_sp500.csv,0.015641,0.000008,0.954696,2081,0.078328,0.077847,0.078328,0.077848,None,0.123498,0.203244,0.099356,0.048897,None
4,monthly_vix_close.csv,0.034492,0.000009,0.386571,4050,0.856296,0.855926,0.856296,0.855968,None,0.598272,0.596333,0.529730,0.539803,None
5,monthly_vix_high.csv,0.040655,0.000010,0.417493,4050,0.856543,0.856050,0.856543,0.856133,None,0.656543,0.521038,0.509061,0.501817,None
6,monthly_vix_low.csv,0.041028,0.000010,0.333374,4050,0.911605,0.911359,0.911605,0.911359,None,0.700000,0.613792,0.601214,0.602729,None
7,monthly_vix_open.csv,0.044954,0.000011,0.408453,4050,0.862963,0.862264,0.862963,0.862388,None,0.554074,0.530883,0.537361,0.518355,None
8,daily_natural_gas.csv,0.045402,0.000008,0.066060,5798,0.997068,0.996905,0.997068,0.996914,None,0.942049,0.313938,0.314351,0.314140,None
9,daily_oil_prices.csv,0.058486,0.000007,0.556240,8300,0.506747,0.506627,0.506747,0.506627,None,0.546627,0.439345,0.437888,0.400675,None


In [9]:
features = [
    'dataset',
    'total-infer-time-tm',
    'infer-time-tm',
    'nupic-anomaly-avg',
    'anomaly-avg-tm',
    'nupic-anomaly-prob-avg',
    'anomaly-samples-tm',
    'accuracy0-tm',
    'precision0-tm',
    'recall0-tm',
    'fscore0-tm',
    'support0-tm',
    'accuracy1-tm',
    'precision1-tm',
    'recall1-tm',
    'fscore1-tm',
    'support1-tm'
]
df[features]

,dataset,total-infer-time-tm,infer-time-tm,nupic-anomaly-avg,anomaly-avg-tm,nupic-anomaly-prob-avg,anomaly-samples-tm,accuracy0-tm,precision0-tm,recall0-tm,fscore0-tm,support0-tm,accuracy1-tm,precision1-tm,recall1-tm,fscore1-tm,support1-tm
0,monthly_sp500.csv,0.264894,0.000150,0.275631,0.275220,0.434315,1765,0.867989,0.861854,0.867989,0.863651,None,0.592635,0.589043,0.560732,0.531980,None
1,weekly_dow_jones.csv,0.622379,0.000299,0.933026,0.932994,0.519712,2079,0.364598,0.349640,0.364598,0.354178,None,0.111111,0.123908,0.105583,0.090165,None
2,weekly_nasdaq.csv,0.725654,0.000349,0.759664,0.759549,0.519942,2080,0.663462,0.645330,0.663462,0.650899,None,0.218269,0.225179,0.213549,0.204291,None
3,weekly_sp500.csv,0.648281,0.000312,0.624306,0.624126,0.520173,2081,0.813551,0.795213,0.813551,0.800885,None,0.296012,0.286399,0.275964,0.273429,None
4,monthly_vix_close.csv,0.628209,0.000155,0.036466,0.036228,0.753394,4050,0.995556,0.995325,0.995556,0.995340,None,0.168642,0.381340,0.277469,0.180174,None
5,monthly_vix_high.csv,1.269400,0.000313,0.073199,0.072970,0.753394,4050,0.993086,0.991989,0.993086,0.992289,None,0.243951,0.253864,0.218215,0.102775,None
6,monthly_vix_low.csv,0.953608,0.000235,0.046971,0.046735,0.753394,4050,0.995802,0.994593,0.995802,0.994942,None,0.156543,0.340974,0.253906,0.162094,None
7,monthly_vix_open.csv,1.444294,0.000357,0.063129,0.062898,0.753394,4050,0.993827,0.992568,0.993827,0.992912,None,0.184444,0.361950,0.235506,0.146606,None
8,daily_natural_gas.csv,0.420894,0.000073,0.010093,0.009922,0.827729,5798,0.997930,0.997774,0.997930,0.997787,None,0.512591,0.203773,0.176054,0.147669,None
9,daily_oil_prices.csv,0.870286,0.000105,0.019382,0.019264,0.879653,8300,0.998795,0.998509,0.998795,0.998581,None,0.513976,0.809178,0.589504,0.589053,None
